In [6]:
import pandas as pd
import pytesseract
from PIL import Image
import re

pytesseract.pytesseract.tesseract_cmd=r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [7]:
dataset_name="gretalai"

In [8]:
disease_data_test = pd.read_csv(f'dataset\\{dataset_name}_dataset_test.csv')
disease_data_train= pd.read_csv(f'dataset\\{dataset_name}_dataset_train.csv')

In [9]:
def get_dataframe_from_images(disease_data):
    reconstructed_data = []
    for _, row in disease_data.iterrows():
        image_path = "dataset\\"+row['file_name']
        disease = row['disease']
        
        img = Image.open(image_path)
        extracted_text = pytesseract.image_to_string(img)
    

        cleaned_text = re.sub(r"Hospital Name:.*", "", extracted_text)  
        cleaned_text = re.sub(r"\n+", "\n", cleaned_text)  
        cleaned_text = cleaned_text.strip() 
        
        cleaned_text = cleaned_text.replace('|', 'I')  
        
        cleaned_text = re.sub(r"[^\x00-\x7F]+", '', cleaned_text)  
    
        if "Symptoms:" in cleaned_text:
            description = cleaned_text.split("Symptoms:")[-1].strip()
        else:
            description = cleaned_text.strip()
        
        description = description.replace('\n', ' ').strip()  
    

        reconstructed_data.append({
            'description': description,
            'disease': disease
        })
    reconstructed_df = pd.DataFrame(reconstructed_data)
    return reconstructed_df

In [10]:
train1=get_dataframe_from_images(disease_data_train)
test1=get_dataframe_from_images(disease_data_test)

In [11]:
train1.to_csv(f'{dataset_name}_train.csv',index=False)
test1.to_csv(f'{dataset_name}_test.csv',index=False)

In [12]:
train1

,description,disease
0,I've been having a lot of pain in my neck and ...,Cervical Spondylosis
1,I have a rash on my face that is getting worse...,Impetigo
2,I have been urinating blood. I sometimes feel ...,Urinary Tract Infection
3,I have been having trouble with my muscles and...,Arthritis
4,I have been feeling really sick. My body hurts...,Dengue
...,...,...
848,My veins are bulging and painful. I can't stan...,Varicose Veins
849,I have been having headaches for a while now. ...,Migraine
850,I have a rash on my face that is very painful ...,Impetigo
851,I have a stuffy nose and nasal congestion. I s...,Allergy


In [13]:
test1

,description,disease
0,I have a burning sensation in my stomach that ...,Peptic Ulcer Disease
1,I have a hard time swallowing and I feel like ...,Peptic Ulcer Disease
2,"I've been having headaches and migraines, and ...",Drug Reaction
3,I'm sweating a lot and can't catch my breath. ...,Pneumonia
4,"I've been scratching myself a lot lately, and ...",Fungal Infection
...,...,...
207,I have been experiencing muscle pain that make...,Dengue
208,"I have red, irritated skin on my arms, face, a...",Psoriasis
209,"I've been having a hard time breathing, and I'...",Bronchial Asthma
210,I've been coughing a lot for a few days now. I...,Bronchial Asthma
